In [1]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.cross_validation import train_test_split
import time

In [2]:
# Path to people.csv from ReadHatKaggle data set
FEATURE_FILE='kaggle-predicting-Red-Hat-Business-Value/Data/act_train_features.csv'
# Path to act_train.csv from RedHatKaggle data set
OUTPUT='kaggle-predicting-Red-Hat-Business-Value/Data/act_train_output.csv'

# For enabling one hot key encoding
ONE_HOT_KEY = True

# Columns that do not represent features
SELECTED_FEATURES = ['char_3', 'char_4', 'char_5', 'char_6',
                     'char_8', 'char_9', 'days', 'month', 
                     'people_char_1', 'people_char_10', 'people_char_11', 
                     'people_char_12', 'people_char_13', 'people_char_14', 
                     'people_char_15', 'people_char_16', 'people_char_17', 
                     'people_char_18', 'people_char_19', 'people_char_2', 
                     'people_char_20', 'people_char_21', 'people_char_22', 
                     'people_char_23', 'people_char_24', 'people_char_25',
                     'people_char_26', 'people_char_27', 'people_char_28', 
                     'people_char_29', 'people_char_3', 'people_char_30', 
                     'people_char_31', 'people_char_32', 'people_char_33', 
                     'people_char_34', 'people_char_35', 'people_char_36',
                     'people_char_37', 'people_char_38', 'people_char_4', 
                     'people_char_5', 'people_char_6', 'people_char_7', 
                     'people_char_8', 'people_char_9', 'people_dayOfMonth', 
                     'people_month', 'people_quarter', 'people_week', 
                     'people_year', 'quarter', 'week', 'year']
# CONT COLUMN
CONT=['people_char_38']

In [3]:
# Function to change labels of categories to one-hot encoding using scikit's OneHot Encoding
# pd.get_dummies(df) does the same, provides sweet header's as well but it it not fast enough, kill's memory
def category_to_one_hot(dataset,non_feature):
    ds = dataset.drop(non_feature,axis=1)
    boolean_column = []
    counter=0
    for column in ds.columns:
        if column not in CONT:
            boolean_column.append(counter)
        counter += 1
    # boolean_colum is not the column name but index
    print ("Done filtering columns...")
    grd_enc = OneHotEncoder(categorical_features=boolean_column)
    encoded_arr=grd_enc.fit_transform(ds).toarray()
    return encoded_arr,grd_enc

In [4]:
# Read the data set's
data_df=pd.read_csv(FEATURE_FILE,parse_dates=["date"])
data_df.sort_values(by=['activity_id'],ascending=True, inplace=True)

IOError: File kaggle-predicting-Red-Hat-Business-Value/Data/act_train_features.csv does not exist

In [ ]:
NON_FEATURES=[]
for column in data_df.columns:
    if column not in SELECTED_FEATURES:
        NON_FEATURES.append(column)

In [ ]:
############################ UNTOUCHED ######################################
def normalize(df,non_features):
    df=df.drop(non_features,axis=1)
    features=df.columns
    # Normalize categorical values to range betwwen 0-1, time usually : 3secs
    start=time.time()
    scaler = Normalizer()
    df[features] = scaler.fit_transform(df[features])
    end = time.time()
    print (end-start)
    return df

In [ ]:
start=time.time()
# Time taken: 120 sec average
arr,enc = category_to_one_hot(data_df,NON_FEATURES)
end = time.time()
print (end-start)

In [ ]:
scal=Normalizer()
arr2=scal.fit_transform(arr)

In [ ]:
# load output
output = pd.read_csv(OUTPUT)
output.sort_values(by='activity_id',ascending=True, inplace=True)
v_out=output['outcome'].as_matrix()

In [ ]:
## Try random forest + linear regression
# forest_data=data_df[[ 'people_group_1']].as_matrix()
X_train, X_test, y_train, y_test = train_test_split(arr,v_out, test_size=0.4, random_state=4)

In [ ]:
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5,
                    colsample_bytree=0.5, seed=0)

xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict_proba(df_test)

In [ ]:
np.savetxt('feature_importance_xgb' + '.txt',
           xgb.feature_importances_, delimiter=',')